In [1]:
import pandas as pd
import os

folder_path = '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data'  # replace with the path to your folder
dfs = []  # initialize an empty list to store the DataFrames

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    dfs.append(file_path)  # append the DataFrame to the list

In [2]:
len(dfs)

11

In [3]:
dfs

['/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/net_exchange.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/prices_brent_lng_updated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/.DS_Store',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/da_production_prognosis.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/electricity_consumption.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/elspot_prices_dt_interpolated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/weather_averaged_preprocessed_NO2.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/reservoir_levels.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/prices_osebx_updated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/wind_settlement.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/electricity_production.csv']

In [4]:
df1=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/elspot_prices_dt_interpolated.csv')
df1.set_index('datetime', inplace=True)
df1 = df1[~df1.index.duplicated()]

In [5]:
df1=df1['Kr.sand']
df1 = df1.to_frame('price')

In [6]:
df2=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/electricity_consumption.csv')
df2.set_index('datetime', inplace=True)
df2 = df2[~df2.index.duplicated()]

In [7]:
df2=df2['NO2']
df2.sort_index(ascending=True, inplace=True)
df2 = df2.to_frame('consumption')

In [8]:
df3=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/electricity_production.csv')
df3.set_index('datetime', inplace=True)
df3 = df3[~df3.index.duplicated()]

In [9]:
df3=df3['NO2']
df3.sort_index(ascending=True, inplace=True)
df3 = df3.to_frame('production')

In [10]:
df4=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/da_production_prognosis.csv')
df4.set_index('datetime', inplace=True)
df4 = df4[~df4.index.duplicated()]

In [11]:
df4=df4['NO2']
df4.sort_index(ascending=True, inplace=True)
df4 = df4.to_frame('da_production')

In [12]:
df5=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/net_exchange.csv')
df5.set_index('datetime', inplace=True)
df5 = df5[~df5.index.duplicated()]

In [13]:
df5.sort_index(ascending=True, inplace=True)
filtered_cols = df5.columns[df5.columns.str.contains('NO2')]
df5=df5[filtered_cols]
df5.drop(columns=['NO2_DE', 'NO2_GB', 'NO2_DK1'], inplace=True)
df5=df5.rename(columns={'NO1_NO2': 'exchange_NO1_NO2', 'NO2_NL':'exchange_NO2_NL', 'NO2_NO5':'exchange_NO2_NO5'})
df5

,exchange_NO1_NO2,exchange_NO2_NL,exchange_NO2_NO5
datetime,,,
2019-12-30 00:00:00,697.0,33.0,-154.0
2019-12-30 01:00:00,726.0,0.0,-104.0
2019-12-30 02:00:00,850.0,1.0,-136.0
2019-12-30 03:00:00,866.0,66.0,-151.0
2019-12-30 04:00:00,776.0,3.0,-152.0
...,...,...,...
2023-04-07 19:00:00,NaN,NaN,NaN
2023-04-07 20:00:00,NaN,NaN,NaN
2023-04-07 21:00:00,NaN,NaN,NaN


In [14]:
df7=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/weather_averaged_preprocessed_NO2.csv')
# Convert the 'date' column to datetime
df7['referenceTime'] = pd.to_datetime(df7['referenceTime'])

# Extract the date part and assign it back to the 'date' column
df7['datetime'] = df7['referenceTime'].dt.date
df7['datetime'] = pd.to_datetime(df7['datetime'])
df7.set_index('datetime', inplace=True)
df7 = df7[~df7.index.duplicated()]
df7=df7.rename(columns={'mean(air_temperature P1D)': 'air_temperature', 'mean(wind_speed P1D)': 'wind_speed','sum(precipitation_amount P1D)':'precipitation_amount' })
df7.drop('referenceTime', axis=1, inplace=True)
df7 = df7.resample('H').ffill()


In [15]:
df8=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/reservoir_levels.csv')
# Convert the 'date' column to datetime
df8['datetime'] = pd.to_datetime(df8['Year/Week'] + '-1', format='%Y/%U-%w')
df8.set_index('datetime', inplace=True)
df8 = df8[~df8.index.duplicated()]
# 1. Convert string dates to datetime objects
start_date = pd.to_datetime('2019-12-29')
end_date = pd.to_datetime('2023-04-01')

# 2. Create new index using pd.date_range()
new_index = pd.date_range(start=start_date, end=end_date, freq='H')

# 3. Reindex DataFrame using new index
df8 = df8.reindex(pd.to_datetime(new_index))

df8 = df8.fillna(method='backfill')
df8 =df8.fillna(method='ffill')
df8=df8[['Capacity_GWh', 'Levels_GWh']]
df8=df8.rename(columns={'Capacity_GWh':'reservoir_capacity', 'Levels_GWh': 'reservoir_levels'})


In [16]:
df9=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/prices_brent_lng_updated.csv')
df9['datetime'] = pd.to_datetime(df9['Date'])
df9.set_index('datetime', inplace=True, drop=True)
df9 = df9[~df9.index.duplicated()]
df9 = df9.resample('H').ffill()
df9=df9[['BZ=F','NG=F']]
df9=df9.rename(columns={'BZ=F':'oil_price','NG=F':'gas_price' })

In [17]:
df10=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO2_data/prices_osebx_updated.csv')
df10['datetime'] = pd.to_datetime(df10['Date'])
df10.set_index('datetime', inplace=True, drop=True)
df10 = df10[~df10.index.duplicated()]
df10 = df10.resample('H').ffill()
df10 = df10.rename(columns={'Adj Close': 'osebx_price'})
df10 =df10['osebx_price']
df10 = df10.to_frame('osebx_price')

In [18]:
dfs = [df1, df2, df3, df4, df5, df7, df8, df9, df10]
df=df1.join(df2, on=df1.index, how='left')
df=df.join(df3,  on=df1.index, how='left')
df=df.join(df4,  on=df1.index, how='left')
df=df.join(df5,  on=df1.index, how='left')

In [19]:
df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')

In [20]:
df=df.join(df7,  on='datetime', how='left')
df=df.join(df8,  on='datetime', how='left')
df=df.join(df9,  on='datetime', how='left')
df=df.join(df10,  on='datetime', how='left')

In [21]:
df

,price,consumption,production,da_production,exchange_NO1_NO2,exchange_NO2_NL,exchange_NO2_NO5,air_temperature,wind_speed,precipitation_amount,reservoir_capacity,reservoir_levels,oil_price,gas_price,osebx_price
datetime,,,,,,,,,,,,,,,
2020-01-01 00:00:00,31386.0,4235.0,4325.0,4647.0,796.0,-433.0,42.0,4.953509,5.021212,0.655738,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 01:00:00,31336.0,4304.0,4865.0,4867.0,86.0,-433.0,386.0,4.953509,5.021212,0.655738,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 02:00:00,31139.0,4139.0,4900.0,4784.0,385.0,-433.0,341.0,4.953509,5.021212,0.655738,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 03:00:00,30853.0,4114.0,4832.0,4734.0,550.0,-433.0,287.0,4.953509,5.021212,0.655738,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 04:00:00,30301.0,4030.0,4616.0,4605.0,721.0,-428.0,188.0,4.953509,5.021212,0.655738,87301.0,55923.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17 19:00:00,117379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-17 20:00:00,117254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-17 21:00:00,117014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df=df.fillna(method='backfill')
df=df.dropna()
df

,price,consumption,production,da_production,exchange_NO1_NO2,exchange_NO2_NL,exchange_NO2_NO5,air_temperature,wind_speed,precipitation_amount,reservoir_capacity,reservoir_levels,oil_price,gas_price,osebx_price
datetime,,,,,,,,,,,,,,,
2020-01-01 00:00:00,31386.0,4235.0,4325.0,4647.0,796.0,-433.0,42.0,4.953509,5.021212,0.655738,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 01:00:00,31336.0,4304.0,4865.0,4867.0,86.0,-433.0,386.0,4.953509,5.021212,0.655738,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 02:00:00,31139.0,4139.0,4900.0,4784.0,385.0,-433.0,341.0,4.953509,5.021212,0.655738,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 03:00:00,30853.0,4114.0,4832.0,4734.0,550.0,-433.0,287.0,4.953509,5.021212,0.655738,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 04:00:00,30301.0,4030.0,4616.0,4605.0,721.0,-428.0,188.0,4.953509,5.021212,0.655738,87301.0,55923.0,582.464933,18.656462,941.270020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29 20:00:00,144605.0,4755.0,8872.0,9128.0,915.0,-15.0,547.0,1.013415,3.594118,7.334234,87301.0,53981.0,815.527912,20.742414,1168.589966
2023-03-29 21:00:00,125662.0,4824.0,9753.0,9252.0,1095.0,-350.0,472.0,1.013415,3.594118,7.334234,87301.0,53981.0,815.527912,20.742414,1168.589966
2023-03-29 22:00:00,116965.0,4719.0,9887.0,9308.0,981.0,-618.0,470.0,1.013415,3.594118,7.334234,87301.0,53981.0,815.527912,20.742414,1168.589966


In [23]:
df.to_csv('NO2_preprocessed')